In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis

/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis


In [3]:
import pandas as pd
from model.LSTM import LSTM_model
from preprocessing.text_preprocessing import Sentences2LemmaWords
from preprocessing.text_preprocessing import GetUnique
from preprocessing.text_preprocessing import setTokenizer
from preprocessing.text_preprocessing import LemmaWords2Seqeunces
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import time
from log.logger import Logger

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
train = pd.read_csv('/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis/5.4.data/train.tsv', sep='\t')
test = pd.read_csv('/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis/5.4.data/test.tsv', sep='\t')

print('train data shape: ', train.shape)
print('test data shape: ', test.shape)
print('----------[train infomation]----------')
print(train.info())
print('----------[test infomation]----------')
print(test.info())

train data shape:  (156060, 4)
test data shape:  (66292, 3)
----------[train infomation]----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   PhraseId    156060 non-null  int64 
 1   SentenceId  156060 non-null  int64 
 2   Phrase      156060 non-null  object
 3   Sentiment   156060 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 4.8+ MB
None
----------[test infomation]----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66292 entries, 0 to 66291
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PhraseId    66292 non-null  int64 
 1   SentenceId  66292 non-null  int64 
 2   Phrase      66292 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.5+ MB
None


In [5]:
train_tokens = Sentences2LemmaWords(train.Phrase)
test_tokens = Sentences2LemmaWords(test.Phrase)

2020-07-14 11:28:26,089[DEBUG|preprocessing, 17] Sentence2LemmaWords: start
/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis/preprocessing/text_preprocessing.py:23: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 23 of the file /content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis/preprocessing/text_preprocessing.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  sentence = BeautifulSoup(sentence).get_text()


239 / 156060 Phrase

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


10700 / 156060 Phrase

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'log'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


...

155973 / 156060 Phrase

2020-07-14 11:30:54,484[DEBUG|preprocessing, 36] Sentence2LemmaWords: finish. 계산 시간 148.40122365951538sec
2020-07-14 11:30:54,489[DEBUG|preprocessing, 17] Sentence2LemmaWords: start


156060 / 156060 Phrase
66260 / 66292 Phrase

2020-07-14 11:31:56,679[DEBUG|preprocessing, 36] Sentence2LemmaWords: finish. 계산 시간 62.19034004211426sec


66292 / 66292 Phrase


In [6]:
unique_words, maxlen = GetUnique(train_tokens)
print(len(unique_words), maxlen)

2020-07-14 11:31:56,693[DEBUG|preprocessing, 44] GetUnique: start
2020-07-14 11:31:56,820[DEBUG|preprocessing, 56] GetUnique: finish. 계산 시간 0.12675070762634277sec


13745 48


In [7]:
target = train.Sentiment
y_target = to_categorical(target)
num_classes = y_target.shape[1]

In [8]:
x_train, x_valid, y_train, y_valid = train_test_split(train_tokens, y_target,test_size=0.2, stratify=y_target)

In [9]:
tokenizer = setTokenizer(x_train, len(unique_words))
x_train = LemmaWords2Seqeunces(x_train, tokenizer, maxlen)
x_valid = LemmaWords2Seqeunces(x_valid, tokenizer, maxlen)
x_test = LemmaWords2Seqeunces(test_tokens, tokenizer, maxlen)

2020-07-14 11:31:57,961[DEBUG|preprocessing, 63] setTokenizer: start
2020-07-14 11:31:58,874[DEBUG|preprocessing, 70] setTokenizer: finish. 계산 시간 0.9122803211212158sec
2020-07-14 11:31:58,876[DEBUG|preprocessing, 77] LemmaWords2Seqeunces: start
2020-07-14 11:32:00,141[DEBUG|preprocessing, 84] LemmaWords2Seqeunces: finish. 계산 시간 1.264997959136963sec
2020-07-14 11:32:00,151[DEBUG|preprocessing, 77] LemmaWords2Seqeunces: start
2020-07-14 11:32:00,760[DEBUG|preprocessing, 84] LemmaWords2Seqeunces: finish. 계산 시간 0.6094930171966553sec
2020-07-14 11:32:00,763[DEBUG|preprocessing, 77] LemmaWords2Seqeunces: start
2020-07-14 11:32:01,370[DEBUG|preprocessing, 84] LemmaWords2Seqeunces: finish. 계산 시간 0.6067347526550293sec


In [10]:
x_test.shape

(66292, 48)

In [20]:
from model.RNN import RNN_model
model = RNN_model(len(unique_words), maxlen, num_classes)

logger = Logger('main')
start_time = time.time()
logger.debug('fit start')

model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=5,batch_size=516, verbose=1)

fit_time = time.time() - start_time
logger.debug('fit end.fit_time{}'.format(fit_time))

2020-07-14 11:44:44,359[DEBUG|RNN_model_generate, 8] RNN_model: start
2020-07-14 11:44:44,510[DEBUG|RNN_model_generate, 19] RNN_model: finish
2020-07-14 11:44:44,514[DEBUG|main, 6] fit start


Epoch 1/5
242/242 [==============================] - 30s 124ms/step - loss: 1.0335 - accuracy: 0.5873 - val_loss: 0.8758 - val_accuracy: 0.6458
Epoch 2/5
242/242 [==============================] - 30s 124ms/step - loss: 0.7629 - accuracy: 0.6897 - val_loss: 0.8385 - val_accuracy: 0.6596
Epoch 3/5
242/242 [==============================] - 30s 124ms/step - loss: 0.6636 - accuracy: 0.7256 - val_loss: 0.8576 - val_accuracy: 0.6588
Epoch 4/5
242/242 [==============================] - 30s 124ms/step - loss: 0.5998 - accuracy: 0.7484 - val_loss: 0.8699 - val_accuracy: 0.6543
Epoch 5/5
242/242 [==============================] - 30s 124ms/step - loss: 0.5543 - accuracy: 0.7637 - val_loss: 0.9091 - val_accuracy: 0.6541


2020-07-14 11:47:16,204[DEBUG|main, 11] fit end.fit_time151.6900656223297


In [21]:
y_pred = model.predict_classes(x_test)

submission = pd.read_csv('/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis/5.4.data/sampleSubmission.csv')

In [22]:
x_test.shape

(66292, 48)

In [23]:
y_pred.shape

(66292,)

In [24]:
submission.shape

(66292, 2)

In [25]:
submission.to_csv('/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis/submission_RNN2.csv')